In [1]:
# Cell 1: Import necessary libraries and set device
import re
import torch
import torchaudio
from datasets import load_dataset, Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
from torch.utils.data import DataLoader
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cuda


In [16]:
# Cell 2: Load the LibriSpeech dev-clean dataset
librispeech_dataset = load_dataset("librispeech_asr", "clean", split="validation")


In [3]:
# Cell 3: Preprocess the text
def preprocess_text(batch):
    batch["text"] = batch["text"].lower()
    batch["text"] = re.sub(r"[^\w\s']", '', batch["text"])
    return batch

librispeech_dataset = librispeech_dataset.map(preprocess_text)


In [4]:
# Cell 4: Resample audio to 16 kHz
def resample(batch):
    audio = batch["audio"]["array"]
    sampling_rate = batch["audio"]["sampling_rate"]
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    batch["audio"]["array"] = resampler(torch.tensor(audio)).numpy()
    batch["audio"]["sampling_rate"] = 16000
    return batch

librispeech_dataset = librispeech_dataset.map(resample)


In [5]:
# Cell 5: Split the dataset into training and validation sets
from sklearn.model_selection import train_test_split
import pandas as pd

# Cell 5: Split the dataset into training and validation sets using the Dataset's built-in method
train_test_split = librispeech_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']



In [6]:
# Cell 7: Define data collator without moving tensors to device
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Process audio
        input_features = [self.processor(feature["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features[0] for feature in features]
        # Process labels
        labels = [self.processor.tokenizer(feature["text"]).input_ids for feature in features]

        # Pad inputs and labels
        input_features = torch.stack(input_features)
        labels = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(l) for l in labels],
            batch_first=True,
            padding_value=self.processor.tokenizer.pad_token_id
        )

        # Replace padding token id's of the labels by -100 so they are ignored in the loss computation
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        batch = {
            "input_features": input_features,  # Do not move to device here
            "labels": labels,                  # Do not move to device here
        }
        return batch


In [8]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")

In [9]:
# Cell 8: Create DataLoaders with pin_memory=True
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=data_collator, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=data_collator, pin_memory=True)


In [11]:
import evaluate
import numpy as np

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Convert to numpy arrays and move to CPU
    if isinstance(pred_ids, torch.Tensor):
        pred_ids = pred_ids.cpu().numpy()
    else:
        pred_ids = np.array(pred_ids)

    if isinstance(label_ids, torch.Tensor):
        label_ids = label_ids.cpu().numpy()
    else:
        label_ids = np.array(label_ids)

    # Replace -100 with the pad token ID
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [12]:
# Import necessary libraries
import torch
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm.auto import tqdm
small_val_dataset = val_dataset.select(range(100))
# Step 1: Set device and clear GPU cache
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: Load the fine-tuned Whisper model on GPU
model1 = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-dev-clean7/checkpoint-22832").to(device)

# Step 3: Define evaluation training arguments
eval_training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned-dev-clean-eval",
    per_device_eval_batch_size=1,      # Keep batch size low to manage GPU memory
    dataloader_num_workers=0,          # Set to 0 to simplify I/O operations
    remove_unused_columns=False,
    predict_with_generate=True,
    fp16=False,                        # Disable fp16 for stability during evaluation
    evaluation_strategy="no",
    disable_tqdm=False,
    logging_dir="./logs-eval",
    logging_steps=10,
    report_to="none"
)

# Step 4: Initialize Trainer
trainer1 = Seq2SeqTrainer(
    model=model1,
    args=eval_training_args,
    eval_dataset=small_val_dataset,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 5: Perform Evaluation with Debug Print Statements
torch.cuda.empty_cache()  # Clear GPU cache
print("Starting evaluation...")

try:
    eval_results = trainer1.evaluate()
    print(f"Final WER: {eval_results['eval_wer']}")
except RuntimeError as e:
    print(f"Runtime error during evaluation: {e}")
    torch.cuda.empty_cache()


Using device: cuda


C:\Users\srinivas\AppData\Local\Temp\ipykernel_24972\2026952759.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer1 = Seq2SeqTrainer(


Starting evaluation...


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/100 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Final WER: 0.20552519732847602


In [17]:
small_val_dataset.save_to_disk("./small_validation_set")


Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
from transformers import MarianMTModel

# Load the fine-tuned MarianMT model from the local directory
model = MarianMTModel.from_pretrained('./nmt tuned')

# Verify the model has been loaded correctly
print(model.config)


MarianConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "./nmt tuned",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "decoder_vocab_size": 59514,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_

In [20]:
# Cell 1: Load the OpenSubtitles English-French Dataset
from datasets import load_dataset
import torch
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
# Load the dataset
dataset = load_dataset("opus100", "en-fr")

# Inspect the dataset
print(dataset)


DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [21]:
# Cell 2: Preprocess the Data
from transformers import MarianTokenizer

# Load the tokenizer
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

# Define source and target languages
SRC_LANG = "en"
TGT_LANG = "fr"

# Preprocessing function
def preprocess_function(examples):
    inputs = [ex[SRC_LANG] for ex in examples['translation']]
    targets = [ex[TGT_LANG] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


c:\Users\srinivas\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [22]:
# Cell 3: Set Up Evaluation Metrics
import evaluate

# Load BLEU metric
bleu = evaluate.load('sacrebleu')

# Metric computation function
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[tokenizer.decode(l, skip_special_tokens=True)] for l in labels]
    result = bleu.compute(predictions=decoded_preds, references=labels)
    return {"bleu": result["score"]}


In [24]:
# Cell 5: Evaluate the Base Model with GPU


# Check if GPU is available


# Split the data
split_datasets = tokenized_datasets['train'].train_test_split(test_size=0.1)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    dataloader_pin_memory=True,  # Ensures better performance with GPUs
)
small_train_dataset = train_dataset.shuffle(seed=42).select(range(30000))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(1000))
# Use a data collator to handle padding during evaluation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model.to(device),  # Send the model to GPU
    args=training_args,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Evaluate the base model
base_metrics = trainer.evaluate()
print(f"Base Model BLEU Score: {base_metrics['eval_bleu']:.2f}")


C:\Users\srinivas\AppData\Local\Temp\ipykernel_24972\3272446186.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/500 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Base Model BLEU Score: 40.38


In [28]:
del model

In [29]:
model1 = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')


In [30]:
# Initialize the trainer
trainer1 = Seq2SeqTrainer(
    model=model1.to(device),  # Send the model to GPU
    args=training_args,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Evaluate the base model
base_metrics = trainer1.evaluate()
print(f"Base Model BLEU Score: {base_metrics['eval_bleu']:.2f}")

C:\Users\srinivas\AppData\Local\Temp\ipykernel_24972\524794908.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer1 = Seq2SeqTrainer(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/500 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Base Model BLEU Score: 38.54


In [27]:
base_metrics = trainer.evaluate()
print(f"Base Model BLEU Score: {base_metrics['eval_bleu']:.2f}")


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/500 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Base Model BLEU Score: 40.38
